In [5]:
from yahoo_oauth import OAuth2
import os
import json
import league, game, team
import yhandler
import json
import objectpath
import pandas as pd
from pandas.io.json import json_normalize

In [7]:
# get our session context ("sc")
oauth = OAuth2(None, None, from_file='oauth2.json')

[2019-08-14 13:58:05,208 DEBUG] [yahoo_oauth.yahoo_oauth.__init__] Checking 
[2019-08-14 13:58:05,209 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] ELAPSED TIME : 337787.01276397705
[2019-08-14 13:58:05,210 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] TOKEN HAS EXPIRED
[2019-08-14 13:58:05,210 DEBUG] [yahoo_oauth.yahoo_oauth.refresh_access_token] REFRESHING TOKEN


In [8]:
# use package to get game
gm = game.Game(oauth, 'nfl')
print(gm)

In [9]:
#use package to get league_id
league_ids = gm.league_ids()
league_id = league_ids[0]
league_id

'390.l.707700'

In [10]:
#get league
lg = gm.to_league(league_id)
print(f"scoring categories: {lg.stat_categories()}")
print(f"standings:{lg.standings()}")
print(f"teams:{lg.teams()}")
print(f"current user team:{lg.team_key()}")
print(f"current week:{lg.current_week()}")

scoring categories: [{'display_name': 'Pass Yds', 'position_type': 'O'}, {'display_name': 'Pass TD', 'position_type': 'O'}, {'display_name': 'Rush Yds', 'position_type': 'O'}, {'display_name': 'Rush TD', 'position_type': 'O'}, {'display_name': 'Rec Yds', 'position_type': 'O'}, {'display_name': 'Rec TD', 'position_type': 'O'}, {'display_name': '2-PT', 'position_type': 'O'}, {'display_name': 'Fum Ret TD', 'position_type': 'O'}, {'display_name': 'FG 0-19', 'position_type': 'K'}, {'display_name': 'FG 20-29', 'position_type': 'K'}, {'display_name': 'FG 30-39', 'position_type': 'K'}, {'display_name': 'FG 40-49', 'position_type': 'K'}, {'display_name': 'FG 50+', 'position_type': 'K'}, {'display_name': 'PAT Made', 'position_type': 'K'}]
standings:['Resident Stevil', 'Dude in Omaha', 'Hair Force Blue', 'Hawkeyes', 'Healthy Scratch', 'KnightHawks', 'Menty', 'Monarchs', 'Redskins', 'War Machine', 'Werkmonsters', 'Wedgie Woty']
teams:[{'name': 'Resident Stevil', 'team_key': '390.l.707700.t.1'}, {'

In [11]:
# get league teams:
teams = lg.teams()
teams

[{'name': 'Resident Stevil', 'team_key': '390.l.707700.t.1'},
 {'name': 'Dude in Omaha', 'team_key': '390.l.707700.t.2'},
 {'name': 'Hair Force Blue', 'team_key': '390.l.707700.t.3'},
 {'name': 'Hawkeyes', 'team_key': '390.l.707700.t.4'},
 {'name': 'Healthy Scratch', 'team_key': '390.l.707700.t.5'},
 {'name': 'KnightHawks', 'team_key': '390.l.707700.t.6'},
 {'name': 'Menty', 'team_key': '390.l.707700.t.7'},
 {'name': 'Monarchs', 'team_key': '390.l.707700.t.8'},
 {'name': 'Redskins', 'team_key': '390.l.707700.t.9'},
 {'name': 'War Machine', 'team_key': '390.l.707700.t.10'},
 {'name': 'Werkmonsters', 'team_key': '390.l.707700.t.11'},
 {'name': 'Wedgie Woty', 'team_key': '390.l.707700.t.12'}]

In [12]:
for d in teams:
    print(d["team_key"])

390.l.707700.t.1
390.l.707700.t.2
390.l.707700.t.3
390.l.707700.t.4
390.l.707700.t.5
390.l.707700.t.6
390.l.707700.t.7
390.l.707700.t.8
390.l.707700.t.9
390.l.707700.t.10
390.l.707700.t.11
390.l.707700.t.12


In [13]:
# testing manual pulling of data
YAHOO_ENDPOINT = 'https://fantasysports.yahooapis.com/fantasy/v2'
uri = 'game/nfl//'

In [14]:
response = oauth.session.get("{}/{}".format(YAHOO_ENDPOINT, uri),params={'format': 'json'})

In [15]:
jresp = response.json()
jresp

{'fantasy_content': {'xml:lang': 'en-US',
  'yahoo:uri': '/fantasy/v2/game/nfl/',
  'game': [{'game_key': '390',
    'game_id': '390',
    'name': 'Football',
    'code': 'nfl',
    'type': 'full',
    'url': 'https://football.fantasysports.yahoo.com/f1',
    'season': '2019',
    'is_registration_over': 0,
    'is_game_over': 0,
    'is_offseason': 0,
    'is_live_draft_lobby_active': 1}],
  'time': '10.612964630127ms',
  'copyright': 'Data provided by Yahoo! and STATS, LLC',
  'refresh_rate': '60'}}

In [16]:
url = "users;use_login=1/games/teams"
response = oauth.session.get("{}/{}".format(YAHOO_ENDPOINT, url),params={'format': 'json'})

In [17]:
# prepare functions
def get_json_dump(url):
    YAHOO_ENDPOINT = 'https://fantasysports.yahooapis.com/fantasy/v2'
    response = oauth.session.get("{}/{}".format(YAHOO_ENDPOINT, url),params={'format': 'json'})
    jresp = response.json()
    return json.dumps(jresp)
def get_json(url):
    YAHOO_ENDPOINT = 'https://fantasysports.yahooapis.com/fantasy/v2'
    response = oauth.session.get("{}/{}".format(YAHOO_ENDPOINT, url),params={'format': 'json'})
    jresp = response.json()
    return jresp
def get_json_t(url):
    YAHOO_ENDPOINT = 'https://fantasysports.yahooapis.com/fantasy/v2'
    response = oauth.session.get("{}/{}".format(YAHOO_ENDPOINT, url),params={'format': 'json'})
    jresp = response.json()
    t = objectpath.Tree(jresp)
    return t

In [18]:
myteamsurl = "users;use_login=1/games/teams"
myteamsjson = get_json(myteamsurl)
# myteams = myteamsjson.jsonify()
myteamsjson

{'fantasy_content': {'xml:lang': 'en-US',
  'yahoo:uri': '/fantasy/v2/users;use_login=1/games/teams',
  'users': {'0': {'user': [{'guid': 'CCOCHI6ILR3M3O6HT5PTS5KN5U'},
     {'games': {'0': {'game': [{'game_key': '390',
          'game_id': '390',
          'name': 'Football',
          'code': 'nfl',
          'type': 'full',
          'url': 'https://football.fantasysports.yahoo.com/f1',
          'season': '2019',
          'is_registration_over': 0,
          'is_game_over': 0,
          'is_offseason': 0,
          'is_live_draft_lobby_active': 1},
         {'teams': {'0': {'team': [[{'team_key': '390.l.707700.t.5'},
              {'team_id': '5'},
              {'name': 'Healthy Scratch'},
              {'is_owned_by_current_login': 1},
              {'url': 'https://football.fantasysports.yahoo.com/f1/707700/5'},
              {'team_logos': [{'team_logo': {'size': 'large',
                  'url': 'https://s.yimg.com/cv/apiv2/default/nfl/nfl_4.png'}}]},
              {'division

In [19]:
myteamsjson["fantasy_content"]["users"]["0"]["user"][1]["games"]["0"]["game"][1]["teams"]["0"]["team"][0][2]["name"]

'Healthy Scratch'

In [20]:
myteamt = get_json_t(myteamsurl)
jfilter = myteamt.execute('$..(team_key,season,code, name)')
print(jfilter)

<generator object filter_dict at 0x0000025313CA5ED0>


In [21]:
for row in jfilter:
    print(row)

{'season': '2019', 'code': 'nfl', 'name': 'Football'}
{'team_key': '390.l.707700.t.5'}
{'name': 'Healthy Scratch'}


In [22]:
league_id = '390.l.707700'
# get my teams: "users;use_login=1/games/teams"
# get standings: f"league/{league_id}/standings"
# get settings: f"league/{league_id}/settings"
# get matchups: f"league/{league_id}/settings"
# get roster: f"team/{team_key}/roster;week={week}"
# get scoreboard: f"league/{league_id}/scoreboard;week={week}"
# get all teams: "users/games/teams"

In [32]:
oneteamurl = f"/league/{league_id}/teams"
onet = get_json(oneteamurl)
onet = onet["fantasy_content"]["league"][1]["teams"]["0"]["team"][0]
onet

[{'team_key': '390.l.707700.t.1'},
 {'team_id': '1'},
 {'name': 'Resident Stevil'},
 [],
 {'url': 'https://football.fantasysports.yahoo.com/f1/707700/1'},
 {'team_logos': [{'team_logo': {'size': 'large',
     'url': 'https://ct.yimg.com/cy/1520/25313688185_4b0ea69cda_192sq.jpg?ct=fantasy'}}]},
 {'division_id': '1'},
 {'waiver_priority': ''},
 {'faab_balance': '100'},
 {'number_of_moves': 0},
 {'number_of_trades': 0},
 {'roster_adds': {'coverage_type': 'week',
   'coverage_value': '1',
   'value': '0'}},
 [],
 {'league_scoring_type': 'head'},
 [],
 [],
 {'has_draft_grade': 0},
 {'auction_budget_total': '200'},
 [],
 {'managers': [{'manager': {'manager_id': '1',
     'nickname': 'Steve W',
     'guid': 'PORZKU7UN5TJHAWJIOGS5FX6FM',
     'is_commissioner': '1',
     'email': 'skwichmer@gmail.com',
     'image_url': 'https://s.yimg.com/ag/images/4542/27255030819_6f4805_64sq.jpg'}}]}]

In [33]:
onet = [item for item in onet if item != []]
onet

[{'team_key': '390.l.707700.t.1'},
 {'team_id': '1'},
 {'name': 'Resident Stevil'},
 {'url': 'https://football.fantasysports.yahoo.com/f1/707700/1'},
 {'team_logos': [{'team_logo': {'size': 'large',
     'url': 'https://ct.yimg.com/cy/1520/25313688185_4b0ea69cda_192sq.jpg?ct=fantasy'}}]},
 {'division_id': '1'},
 {'waiver_priority': ''},
 {'faab_balance': '100'},
 {'number_of_moves': 0},
 {'number_of_trades': 0},
 {'roster_adds': {'coverage_type': 'week',
   'coverage_value': '1',
   'value': '0'}},
 {'league_scoring_type': 'head'},
 {'has_draft_grade': 0},
 {'auction_budget_total': '200'},
 {'managers': [{'manager': {'manager_id': '1',
     'nickname': 'Steve W',
     'guid': 'PORZKU7UN5TJHAWJIOGS5FX6FM',
     'is_commissioner': '1',
     'email': 'skwichmer@gmail.com',
     'image_url': 'https://s.yimg.com/ag/images/4542/27255030819_6f4805_64sq.jpg'}}]}]

In [34]:
onetdict = {k: v for d in allt for k, v in d.items()}
onetdict

{'team_key': '390.l.707700.t.1',
 'team_id': '1',
 'name': 'Resident Stevil',
 'url': 'https://football.fantasysports.yahoo.com/f1/707700/1',
 'team_logos': [{'team_logo': {'size': 'large',
    'url': 'https://ct.yimg.com/cy/1520/25313688185_4b0ea69cda_192sq.jpg?ct=fantasy'}}],
 'division_id': '1',
 'waiver_priority': '',
 'faab_balance': '100',
 'number_of_moves': 0,
 'number_of_trades': 0,
 'roster_adds': {'coverage_type': 'week', 'coverage_value': '1', 'value': '0'},
 'league_scoring_type': 'head',
 'has_draft_grade': 0,
 'auction_budget_total': '200',
 'managers': [{'manager': {'manager_id': '1',
    'nickname': 'Steve W',
    'guid': 'PORZKU7UN5TJHAWJIOGS5FX6FM',
    'is_commissioner': '1',
    'email': 'skwichmer@gmail.com',
    'image_url': 'https://s.yimg.com/ag/images/4542/27255030819_6f4805_64sq.jpg'}}]}

In [35]:
# can't iterate directly over dictionary
for k,v in onetdict.items():
    if isinstance(onetdict[k], list):
        print("found a list")
        print(onetdict[k][0])
        onetdict[k] = onetdict[k][0]
onetdict

found a list
{'team_logo': {'size': 'large', 'url': 'https://ct.yimg.com/cy/1520/25313688185_4b0ea69cda_192sq.jpg?ct=fantasy'}}
found a list
{'manager': {'manager_id': '1', 'nickname': 'Steve W', 'guid': 'PORZKU7UN5TJHAWJIOGS5FX6FM', 'is_commissioner': '1', 'email': 'skwichmer@gmail.com', 'image_url': 'https://s.yimg.com/ag/images/4542/27255030819_6f4805_64sq.jpg'}}


{'team_key': '390.l.707700.t.1',
 'team_id': '1',
 'name': 'Resident Stevil',
 'url': 'https://football.fantasysports.yahoo.com/f1/707700/1',
 'team_logos': {'team_logo': {'size': 'large',
   'url': 'https://ct.yimg.com/cy/1520/25313688185_4b0ea69cda_192sq.jpg?ct=fantasy'}},
 'division_id': '1',
 'waiver_priority': '',
 'faab_balance': '100',
 'number_of_moves': 0,
 'number_of_trades': 0,
 'roster_adds': {'coverage_type': 'week', 'coverage_value': '1', 'value': '0'},
 'league_scoring_type': 'head',
 'has_draft_grade': 0,
 'auction_budget_total': '200',
 'managers': {'manager': {'manager_id': '1',
   'nickname': 'Steve W',
   'guid': 'PORZKU7UN5TJHAWJIOGS5FX6FM',
   'is_commissioner': '1',
   'email': 'skwichmer@gmail.com',
   'image_url': 'https://s.yimg.com/ag/images/4542/27255030819_6f4805_64sq.jpg'}}}

In [36]:
raw_team_data = json_normalize(onetdict, sep="_")
raw_team_data
# df = pd.DataFrame(alldict)
# df

,auction_budget_total,division_id,faab_balance,has_draft_grade,league_scoring_type,managers_manager_email,managers_manager_guid,managers_manager_image_url,managers_manager_is_commissioner,managers_manager_manager_id,...,number_of_trades,roster_adds_coverage_type,roster_adds_coverage_value,roster_adds_value,team_id,team_key,team_logos_team_logo_size,team_logos_team_logo_url,url,waiver_priority
0,200,1,100,0,head,skwichmer@gmail.com,PORZKU7UN5TJHAWJIOGS5FX6FM,https://s.yimg.com/ag/images/4542/27255030819_...,1,1,...,0,week,1,0,1,390.l.707700.t.1,large,https://ct.yimg.com/cy/1520/25313688185_4b0ea6...,https://football.fantasysports.yahoo.com/f1/70...,


In [55]:
allteamurl = f"/league/{league_id}/teams"
allt = get_json(allteamurl)
allt = allt["fantasy_content"]["league"][1]["teams"]#["0"]["team"][0]
allt

{'0': {'team': [[{'team_key': '390.l.707700.t.1'},
    {'team_id': '1'},
    {'name': 'Resident Stevil'},
    [],
    {'url': 'https://football.fantasysports.yahoo.com/f1/707700/1'},
    {'team_logos': [{'team_logo': {'size': 'large',
        'url': 'https://ct.yimg.com/cy/1520/25313688185_4b0ea69cda_192sq.jpg?ct=fantasy'}}]},
    {'division_id': '1'},
    {'waiver_priority': ''},
    {'faab_balance': '100'},
    {'number_of_moves': 0},
    {'number_of_trades': 0},
    {'roster_adds': {'coverage_type': 'week',
      'coverage_value': '1',
      'value': '0'}},
    [],
    {'league_scoring_type': 'head'},
    [],
    [],
    {'has_draft_grade': 0},
    {'auction_budget_total': '200'},
    [],
    {'managers': [{'manager': {'manager_id': '1',
        'nickname': 'Steve W',
        'guid': 'PORZKU7UN5TJHAWJIOGS5FX6FM',
        'is_commissioner': '1',
        'email': 'skwichmer@gmail.com',
        'image_url': 'https://s.yimg.com/ag/images/4542/27255030819_6f4805_64sq.jpg'}}]}]]},
 '1': {

In [57]:
# allt = allt["fantasy_content"]["league"][1]["teams"]#["0"]["team"][0]
teamlist = []
for k, v in allt.items():
    if k != "count": #ignore count item which json adds as autosummary at the end
        team = allt[k]["team"][0] #go down to individual team level in json
        team = [item for item in team if item != []]     #remove all empty lists
        teamd = { k : v[0] if type(v)==list else v  for d in team for k, v in d.items()} #convert all dictionaries into 1 dictionary using comprehension, if they contain a list only use first item from list
        print(teamd)
        teamlist.append(teamd)
    
#     onet = [item for item in onet if item != []] #remove blank lists
# onetdict = {k: v for d in allt for k, v in d.items()}
# # can't iterate directly over dictionary
# for k,v in onetdict.items():
#     if isinstance(onetdict[k], list):
#         print("found a list")
#         print(onetdict[k][0])
#         onetdict[k] = onetdict[k][0]
# onetdict
    

{'team_key': '390.l.707700.t.1', 'team_id': '1', 'name': 'Resident Stevil', 'url': 'https://football.fantasysports.yahoo.com/f1/707700/1', 'team_logos': {'team_logo': {'size': 'large', 'url': 'https://ct.yimg.com/cy/1520/25313688185_4b0ea69cda_192sq.jpg?ct=fantasy'}}, 'division_id': '1', 'waiver_priority': '', 'faab_balance': '100', 'number_of_moves': 0, 'number_of_trades': 0, 'roster_adds': {'coverage_type': 'week', 'coverage_value': '1', 'value': '0'}, 'league_scoring_type': 'head', 'has_draft_grade': 0, 'auction_budget_total': '200', 'managers': {'manager': {'manager_id': '1', 'nickname': 'Steve W', 'guid': 'PORZKU7UN5TJHAWJIOGS5FX6FM', 'is_commissioner': '1', 'email': 'skwichmer@gmail.com', 'image_url': 'https://s.yimg.com/ag/images/4542/27255030819_6f4805_64sq.jpg'}}}
{'team_key': '390.l.707700.t.2', 'team_id': '2', 'name': 'Dude in Omaha', 'url': 'https://football.fantasysports.yahoo.com/f1/707700/2', 'team_logos': {'team_logo': {'size': 'large', 'url': 'https://s.yimg.com/cv/api

In [62]:
teamsdf = json_normalize(teamlist, sep="_")
teamsdf

# dfteams = pd.DataFrame(teamlist)
# dfteams

,auction_budget_total,division_id,faab_balance,has_draft_grade,is_owned_by_current_login,league_scoring_type,managers_manager_email,managers_manager_guid,managers_manager_image_url,managers_manager_is_commissioner,...,number_of_trades,roster_adds_coverage_type,roster_adds_coverage_value,roster_adds_value,team_id,team_key,team_logos_team_logo_size,team_logos_team_logo_url,url,waiver_priority
0,200,1,100,0,NaN,head,skwichmer@gmail.com,PORZKU7UN5TJHAWJIOGS5FX6FM,https://s.yimg.com/ag/images/4542/27255030819_...,1,...,0,week,1,0,1,390.l.707700.t.1,large,https://ct.yimg.com/cy/1520/25313688185_4b0ea6...,https://football.fantasysports.yahoo.com/f1/70...,
1,200,2,100,0,NaN,head,dannowak27@yahoo.com,MWR6YBRX3HYH7PVV4N77XROVUM,https://ct.yimg.com/cy/1768/39361574426_98028a...,NaN,...,0,week,1,0,2,390.l.707700.t.2,large,https://s.yimg.com/cv/apiv2/default/nfl/nfl_1.png,https://football.fantasysports.yahoo.com/f1/70...,
2,200,3,100,0,NaN,head,NaN,ARWYXQDM7WRKRW4GRBUYOPSD6M,https://ct.yimg.com/cy/1768/39361574426_98028a...,NaN,...,0,week,1,0,3,390.l.707700.t.3,large,https://s.yimg.com/cv/apiv2/default/nfl/nfl_9.png,https://football.fantasysports.yahoo.com/f1/70...,
3,200,1,100,0,NaN,head,r_thompson2@yahoo.com,MO3L5PCUH43HUKFAVZ3KJZG5EM,https://ct.yimg.com/cy/1768/39361574426_98028a...,NaN,...,0,week,1,0,4,390.l.707700.t.4,large,https://ct.yimg.com/cy/1448/25312352535_a61bc4...,https://football.fantasysports.yahoo.com/f1/70...,
4,200,2,100,0,1.0,head,josephghouser@gmail.com,AUW2GOYE3XF4UMXBJLOUWHOLGU,https://ct.yimg.com/cy/1768/39361574426_98028a...,NaN,...,0,week,1,0,5,390.l.707700.t.5,large,https://s.yimg.com/cv/apiv2/default/nfl/nfl_4.png,https://football.fantasysports.yahoo.com/f1/70...,
5,200,3,100,0,NaN,head,matthew.j.angeli@gmail.com,SAXYS3NRVSAKTJAOIU5GCTGFIU,https://ct.yimg.com/cy/1768/39361574426_98028a...,NaN,...,0,week,1,0,6,390.l.707700.t.6,large,https://ct.yimg.com/cy/1788/29709027627_60137a...,https://football.fantasysports.yahoo.com/f1/70...,
6,200,1,100,0,NaN,head,menty84@yahoo.com,WIDAMSGF7B5SZ6UHTHTGJUMXHY,https://ct.yimg.com/cy/1768/39361574426_98028a...,NaN,...,0,week,1,0,7,390.l.707700.t.7,large,https://s.yimg.com/cv/apiv2/default/nfl/nfl_11...,https://football.fantasysports.yahoo.com/f1/70...,
7,200,2,100,0,NaN,head,rightofcenter77@gmail.com,MD7S736DXN5UGVY3IXTFOVLEQA,https://ct.yimg.com/cy/4464/23765584287_6f91df...,NaN,...,0,week,1,0,8,390.l.707700.t.8,large,https://s.yimg.com/cv/apiv2/default/nfl/nfl_10...,https://football.fantasysports.yahoo.com/f1/70...,
8,200,3,100,0,NaN,head,twichmer@sbcglobal.net,U3GCI2MQVCCLSTB5YNG5W6GYJU,https://s.yimg.com/ag/images/4692/37641303913_...,NaN,...,0,week,1,0,9,390.l.707700.t.9,large,https://ct.yimg.com/cy/8359/29274086040_db88cd...,https://football.fantasysports.yahoo.com/f1/70...,
9,200,1,100,0,NaN,head,jameskahre@hotmail.com,7LRVMIBL4UXBP4W7O3M5WQY3C4,https://s.yimg.com/ag/images/4575/38050371440_...,NaN,...,0,week,1,0,10,390.l.707700.t.10,large,https://ct.yimg.com/cy/4439/36042743854_8eb118...,https://football.fantasysports.yahoo.com/f1/70...,


In [ ]:
allteamst = get_json_t(allteamsurl)
allteams = allt["fantasy_content"]["league"][1]["teams"]
mydictlist = []
for key in allteams:
    print(key)
    print(allteams[key]["team"])
    mydictlist.append(allteams[key]["team"])
mydictlist

In [ ]:
df = pd.DataFrame(mydictlist) 
df.head()

In [ ]:
jfilter = allteamst.execute('$..(name)')

filter = f"$.fantasy_content.league.({})[0]"
filter2 = allteamst.execute('$..league')
for row in filter2:
    print(row)
    print("next team_________________________________________________")